In [1]:
from scipy.io import loadmat
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from smooth import smoothen


%matplotlib inline

In [2]:
filelist = ["calcium_data/140708B_140811a_result", 
            "calcium_data/140909C_141112a_result", 
            "calcium_data/141006C_141121a_result",
            "calcium_data/150109A_150302a_result", 
            "calcium_data/151122A_160202a_result", 
            "calcium_data/151122B_160207a_result", 
            "calcium_data/160209A_160430a_result", 
            "calcium_data/160209B_160428a_result"]

experiemntal parameters:

v(0): hit/miss, +1 for hit, -1 for miss

v(1): stimulus intensity

v(2): rest of variance

In [26]:
def preprocessing(filename, time='full', windows=None, z_score=True, smooth=True):
    '''
    Preprocesses the data according to specified parameters
    
    filename: string, specifies file to use. If None, concatenate all files
    time: string
        'full' - use full time series of the dendritic Ca2+ signal
        'windowed' - use only one point per dendrite, averaged over specified window
    windows: ndarray, 7x2 or n_filesx7x2. 
        Required in windowed mode. Array holds start- and stop times for each stimulus.
    z_score: bool, whether to z-score the data. 
    '''
    
    assert(not(time=='windowed') or not(windows == None))
    
    #get the data we need into the format we need
    if type(filename) == list:
        #create concatenated data, prompt if file already exitsts
        #in this case, other experiments are interpreted as different units, but same trials as
        #other units. Not all experiments have same units, and not same stimuli, so...later.
        try:
            f = h5py.File('allexp.hdf5', 'w-')
        except OSError:
            print('file already exists, type "continue" to overwrite \n')
            if input() == 'continue':
                f = h5py.File('allexp.hdf5', 'w')
                
        #filename denotes a list of files in this case
        for file in filename:
            #finish later, or probably not
            pass
    else:
        #just load data from specified file
        f = h5py.File(filename+'.hdf5', 'r')
        g = h5py.File(filename[:-6]+"roi.hdf5", "r")
        motion_mask = (g['inFrameDend'][:].astype(bool)).reshape(g['inFrameDend'].shape[0])
        
        data = f['data'][:, motion_mask, :]
        meta = f['meta']
    
    if time == 'full':
        #keep all times
        data_pass = data[:]
        if smooth:
            #if we smoothen the data
            data_pass = smoothen(filename)[:,motion_mask,:]
        meta_pass = meta[:]
        
    if time == 'windowed':
        #get window-averaged
        meta_pass = meta[:]
        data_pass = np.zeros((data.shape[0], data.shape[1]))
        
        stims = np.unique(meta[:,1])
        for i, amp in enumerate(stims):
            amp_mask = meta[:,1] == amp
            data_pass[amp_mask] = np.mean(data[amp_mask,:,windows[i,0], windows[i,1]], axis=2)
            
    if z_score:
        #z_score signal
        mns = np.mean(data_pass, axis=0)
        try:
        #specify exception
            mns = np.mean(mns, axis=1)
        except:
            print('Exception triggered')
            
        try:
            stds = np.std(data_pass, axis=(0,2))
        except:
            print('Exception triggered')
            
        mns = np.transpose(np.tile(mns, (data.shape[0], data.shape[2], 1)), axes=[0,2,1])
        stds = np.transpose(np.tile(stds, (data.shape[0], data.shape[2], 1)), axes=[0,2,1])
        
        data_pass = data_pass - mns
        data_pass = data_pass/stds
            
    #possibly add functionality for subtracting mean, but not dividing by std.
    return meta_pass, data_pass

In [132]:
def get_betas(meta, data):
    n_trials = data.shape[0]
    n_units = data.shape[1]
    n_times = data.shape[2]
    
    F_i = np.ones((3, n_trials))
    F_i[0, :] = meta[:, 2]
    F_i[1, :] = meta[:, 1]
    
    #tensor math more efficient?
    beta = np.zeros((n_units, n_times, 3))
    F_fac = np.dot(np.linalg.inv(np.dot(F_i, F_i.T)), F_i)
    
    for i in range(n_units):
        for t in range(n_times):
            r_it = data[:,i,t]
            beta[i, t, :] = F_fac.dot(r_it)
            
    return beta

In [207]:
def get_population_response(meta, data):
    '''
    returns ndarray of shape conditions x times x units
    '''
    n_units = data.shape[1]
    n_trials = data.shape[0]
    n_times = data.shape[2]
    
    stims = np.unique(meta[:,1])
    n_stims = stims.shape[0]
    
    #get mask for hit- and miss conditions
    hit_mask = meta[:, 2] == 1
    miss_mask = meta[:, 2] == 0
    
    hit_stim_masks = []
    miss_stim_masks = []
    
    #get masks for all stimulus conditions
    for stim in stims:
        stim_mask = meta[:, 1] == stim
        hit_stim_masks.append(np.logical_and(stim_mask, hit_mask))
        miss_stim_masks.append(np.logical_and(stim_mask, miss_mask))
        
    n_conditions = len(hit_stim_masks) + len(miss_stim_masks)

    x = np.zeros((n_conditions, n_times, n_units))
    #CONVENTION: all hit, then all miss
    
    for i in range(n_conditions):
        if i < len(hit_stim_masks):
            x[i, :, :] = np.mean(data[hit_stim_masks[i]], axis=0).T
        else:
            i = i%len(hit_stim_masks)
            x[i, :, :] = np.mean(data[miss_stim_masks[i]], axis=0).T
            
    return x

In [173]:
def build_X_PCA():
    pass

In [174]:
meta, data = preprocessing(filelist[0], time='full', windows=None, z_score=True, smooth=True)

In [175]:
beta = get_betas(meta, data)

In [198]:
x = get_population_response(meta, data)

In [206]:
x[1,3]

array([-0.14849715,  0.28754762, -0.04867248, -0.07447999, -0.06215255,
       -0.1042335 ,  0.02175858, -0.07586941, -0.18940984,  0.05272575,
       -0.05978899, -0.17362823, -0.24208255,  0.30556657, -0.13914806,
       -0.02655665,  0.06397083, -0.15002965, -0.01481979,  0.06730582,
        0.19044269,  0.05450866, -0.24650156, -0.3023079 , -0.19561972,
        0.00427318,  0.08992673, -0.02306731,  0.19891316, -0.12063751,
        0.0618432 ,  0.02906641,  0.26620874,  0.04166446, -0.00549963,
        0.14641239, -0.22778411, -0.15444401, -0.25007022, -0.18399296,
        0.02910131,  0.0179669 ,  0.10009634, -0.05261634, -0.21726763,
        0.12593191, -0.01440529, -0.32740382,  0.02243236, -0.05981341,
       -0.15144596,  0.10418613, -0.28803781, -0.30202223, -0.01352433,
       -0.11030802, -0.048679  , -0.20885005,  0.03110869, -0.05085603,
       -0.14285281,  0.08259193, -0.01193314, -0.16655303,  0.24061088,
       -0.03896131,  0.23778656,  0.1444806 , -0.2932495 , -0.10